In [3]:
import pdfplumber
import re
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import fnmatch
import logging
import time
import json
import numpy as np
import os
import joblib
import tensorflow as tf

import lib_plot
import xmltodict
import base64

from timeit import default_timer as timer
import magic
import yaml
import logging
import logging.config


from lib_extraction import create_fus, from_xml_to_mrv_format, from_pdf_to_mrv_format, plumber_df


import xmltodict

In [4]:
PATH = "/home/robin/Downloads/CDA_20200511155057576_MATERIOVIGILANCE.xml"
with open(PATH) as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [15]:
res = pd.DataFrame()
E = []
for elt in data_dict['ClinicalDocument']['component']['structuredBody']['component']:
    for el in elt['section']['text']['table']['tbody']['tr']:
        if type(el)!=str :
            for i in range(len(el['td'])):
                try:
                    res[el['td'][2*i]['#text']] = [el['td'][2*i+1]['#text']]
                except:
                    try:
                        E.append(el['td'][i]['#text'])
                    except:
                        E.append('erreur !')

In [16]:
res

,Nom (3 premières lettres),Sexe,Type de DM,Libellé du produit,Référence commerciale,Fabricant : Nom,Type d'utilisateur du DM,Date de survenue,Type de lieu de survenue,Nom de l'etablissement,...,Service,Personne à contacter dans l'établissement,Téléphone du contact,Description de l'incident,Nombre de patients ou personnes concernées,Nombre de dispositifs concernés,Ce type d’incident est-il prévu dans la notice d’utilisation du fabricant ?,Classification de l'incident,Conséquences cliniques et état actuel du patient ou personne impliquée,Actions entreprises pour le patient vis-à-vis du DM
0,test,F,Masques FFP2,Masque FFP2 Op Air Pro,M52210-WH,KOLMI,Professionnels de santé,30/04/2020,Etablissement de santé,Centre Hospitalier d'Alençon-Mamers,...,UHCD,XXXXX XXXXX,0233323020,les élastiques du masque FFP2 cassent lors de ...,1,1,NSP,Menace importante pour la santé publique,Implique l'exposition directe du soignant à un...,Consommation d'un nouveau masque FFP2


In [18]:
import re

filename = PATH.split('/')[-1].lower()
if ('matériovigilance' in filename) or ('materiovigilance' in filename):
    TYPE = 'Matériovigilance'
elif ('réactovigilance' in filename) or ('réactovigilance' in filename):
    TYPE = 'Réactovigilance'
else :           
    TYPE = ''

number = re.findall(r'\d+', filename)
if len(number)>0:
    NUMERO = number[0]
else : 
    NUMERO = ''

In [20]:
NUMERO

'20200511155057576'